In [ ]:
import logging
from pathlib import Path

# Create a cartopy plot with CRS is EPSG:3413 (Focus on north pole)
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import hvplot.xarray
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import numpy as np
import rioxarray
import rioxarray.merge
import xarray as xr
import zarr
from darts_acquisition.arcticdem.datacube import download_arcticdem_extent
from dask.distributed import Client
from rich import traceback
from rich.logging import RichHandler

from darts.utils.logging import LoggingManager

LoggingManager.setup_logging()
logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)],
)
traceback.install(show_locals=False)
client = Client()
client

In [ ]:
DATA_ROOT = Path("../data")

fpath = DATA_ROOT / "input/planet/PSOrthoTile/4974017/5854937_4974017_2022-08-14_2475"
arcticdem_dir = DATA_ROOT / "download/arcticdem"
resolution = 32
coarsen = 10

In [ ]:
extent_fpath = arcticdem_dir / f"ArcticDEM_Mosaic_Index_v4_1_{resolution}m.parquet"
if not extent_fpath.exists():
    download_arcticdem_extent(arcticdem_dir)
extent = gpd.read_parquet(extent_fpath)
extent.head()

In [ ]:
datacube_fpath = arcticdem_dir / f"datacube_{resolution}m_v4.1.zarr"
storage = zarr.storage.FSStore(datacube_fpath)

# Check if zarr data already contains the data via the attrs
arcticdem_datacube = xr.open_zarr(storage, mask_and_scale=True)
loaded_scenes = arcticdem_datacube.attrs.get("loaded_scenes", []).copy()
# extent[extent.dem_id.isin(loaded_scenes)].explore()

In [ ]:
allbounds = extent[extent.dem_id.isin(loaded_scenes)].bounds
bounds = {
    "x": slice(allbounds.minx.min(), allbounds.maxx.max()),
    "y": slice(allbounds.maxy.max(), allbounds.miny.min()),
}
arcticdem_datacube_subset = arcticdem_datacube.sel(bounds)
arcticdem_datacube_subset

In [ ]:
import hvplot.xarray  # noqa

# crs = ccrs.NorthPolarStereo(central_longitude=-45)
crs = arcticdem_datacube.rio.crs

# Calc height / width
height = 800  # px
width = int(height * arcticdem_datacube_subset.dem.shape[1] / arcticdem_datacube_subset.dem.shape[0])

arcticdem_datacube_subset.hvplot.image(
    x="x",
    y="y",
    z="dem",
    rasterize=True,
    aggregator="max",
    crs=crs,
    projection=crs,
    cmap="gray",
    dynamic=True,
    colorbar=True,
    height=height,
    width=width,
)

In [ ]:
lowres_data = []
for sceneinfo in extent.itertuples():
    if sceneinfo.dem_id not in loaded_scenes:
        continue

    if "SHAPE" in sceneinfo._fields:
        geom = sceneinfo.SHAPE
    elif "geom" in sceneinfo._fields:
        geom = sceneinfo.geom
    else:
        geom = sceneinfo.geometry
    scene_data = arcticdem_datacube.dem.sel(
        x=slice(geom.bounds[0], geom.bounds[2]), y=slice(geom.bounds[3], geom.bounds[1])
    )
    if coarsen is not None:
        scene_data_lowres = scene_data.coarsen(x=coarsen, y=coarsen, boundary="trim").mean()
        lowres_data.append(scene_data_lowres)
    else:
        lowres_data.append(scene_data)
print(f"Already {len(lowres_data)} tiles downloaded for {resolution=}m")
lowres_data_da = rioxarray.merge.merge_arrays([da.rio.write_crs(3413) for da in lowres_data], nodata=float("nan"))
lowres_data_da

In [ ]:
# Define the projection
projection = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70)

# Create a figure
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={"projection": projection})

# Set the extent to focus on the North Pole
ax.set_extent([-180, 180, 60, 90], crs=ccrs.PlateCarree())

# Add features
ax.add_feature(cfeature.LAND, zorder=0, edgecolor="black", facecolor="white")
ax.add_feature(cfeature.OCEAN, zorder=0, facecolor="lightgrey")
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=":")
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.RIVERS)

# Add gridlines
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Compute a circle in axes coordinates, which we can use as a boundary
# for the map. We can pan/zoom as much as we like - the boundary will be
# permanently circular.
theta = np.linspace(0, 2 * np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

ax.set_boundary(circle, transform=ax.transAxes)

# Add the xarray data
lowres_data_da.rio.reproject("epsg:4326").plot(ax=ax, transform=ccrs.PlateCarree(), cmap="viridis", add_colorbar=False)

# Show the plot
plt.show()

## Compare loaded extends

In [ ]:
import folium

resolutions = [2, 10, 32]

extents = {}
for resolution in resolutions:
    # Load extent file of arcticdem
    extent_fpath = arcticdem_dir / f"ArcticDEM_Mosaic_Index_v4_1_{resolution}m.parquet"
    if not extent_fpath.exists():
        download_arcticdem_extent(arcticdem_dir, resolution=resolution)
    extent = gpd.read_parquet(extent_fpath)
    extents[resolution] = extent

loaded_extents = {}
for resolution in resolutions:
    datacube_fpath = arcticdem_dir / f"datacube_{resolution}m_v4.1.zarr"
    storage = zarr.storage.FSStore(datacube_fpath)
    arcticdem_datacube = xr.open_zarr(storage, mask_and_scale=True)
    loaded_scenes = arcticdem_datacube.attrs.get("loaded_scenes", []).copy()

    extent = extents[resolution]
    loaded_extent = extent[extent.dem_id.isin(loaded_scenes)].copy()
    loaded_extents[resolution] = loaded_extent

m = folium.Map()
for resolution in resolutions:
    extents[resolution].to_crs("EPSG:4326").explore(m=m, name=f"ArcticDEM {resolution}m", color="blue")
    loaded_extents[resolution].to_crs("EPSG:4326").explore(m=m, name=f"ArcticDEM {resolution}m (loaded)", color="red")

folium.LayerControl().add_to(m)
m